# Getting Data Ready For Model

In [12]:
%run src/imports.py

In [ ]:
%run src/getDF.py

/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-54-03.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-04-11-49-33.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-14-23.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-47-54.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-04-11-50-11.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-27-39.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-20-18.csv
/Users/randallbloomquist/Desktop/Desktop/Galvanize/CAPSTONES/Sell Your Home in 30 days/data/redfin_2021-03-14-18-26-51.csv
/Users/randallbl

In [4]:
df = pd.read_pickle('data/full.pkl')
df.drop_duplicates(inplace=True)

In [6]:
df['ROOMS'] = df['BEDS'] + df['BATHS']
df = df.drop(['$/SQUARE FEET','BEDS','BATHS'],axis=1)

In [7]:
counts = df.groupby('LOCATION').count()
neighborhoods = counts[counts['SOLD DATE'] > 50].index
neighborhoods_cleaned = df[df['LOCATION'].isin(neighborhoods)]
neighborhoods_cleaned.to_pickle('data/neighborhoods_cleaned.pkl')

In [8]:
neighborhoods_cleaned = pd.read_pickle('data/neighborhoods_cleaned.pkl')
neighborhoods_cleaned.reset_index(inplace=True)
neighborhoods_cleaned.drop(['index','DAYS ON MARKET','PROPERTY TYPE','ADDRESS','CITY','STATE OR PROVINCE'],axis=1,inplace=True)
#neighborhoods_cleaned = neighborhoods_cleaned.merge(pd.get_dummies(neighborhoods_cleaned['LOCATION'],drop_first=True), how='outer',left_index=True,right_index=True)
neighborhoods_cleaned.drop(['LOCATION'],axis=1,inplace=True)
neighborhoods_cleaned['SOLD DATE'] = pd.to_datetime(neighborhoods_cleaned['SOLD DATE'])
neighborhoods_cleaned['MONTH'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).month
neighborhoods_cleaned['YEAR'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).year
neighborhoods_cleaned.drop(['SOLD DATE'],axis=1,inplace=True)

In [9]:
neighborhoods_cleaned

,SALE TYPE,ZIP OR POSTAL CODE,PRICE,SQUARE FEET,LOT SIZE,YEAR BUILT,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,ROOMS,MONTH,YEAR
0,PAST SALE,78257,345000,1989.0,5662.0,2005.0,230.0,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/6702-Frua...,Sabor,1467693.0,N,Y,29.647801,-98.614186,4.0,9,2020
1,PAST SALE,78256,337500,2166.0,9583.0,2004.0,54.0,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/315-Persi...,Sabor,1451684.0,N,Y,29.650304,-98.629082,6.0,6,2020
2,PAST SALE,78256,375500,2969.0,6605.0,2004.0,54.0,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/610-Cypre...,Sabor,1496790.0,N,Y,29.651842,-98.630650,6.5,3,2021
3,PAST SALE,78255,462000,2716.0,9147.0,2014.0,95.0,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/18342-Bra...,Sabor,1329092.0,N,Y,29.617650,-98.643430,7.0,10,2018
4,PAST SALE,78256,624000,3012.0,75794.0,2000.0,25.0,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/6611-Sier...,Sabor,1501429.0,N,Y,29.620646,-98.622295,5.5,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20007,PAST SALE,78222,153000,1577.0,6969.0,1983.0,NaN,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/5926-Whis...,Sabor,1400891.0,N,Y,29.383410,-98.376520,5.0,9,2019
20008,PAST SALE,78222,189900,1703.0,4791.0,2018.0,25.0,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/5510-Goli...,Sabor,1305836.0,N,Y,29.386700,-98.379900,6.0,7,2018
20009,PAST SALE,78222,128000,1102.0,5662.0,1985.0,NaN,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/5931-Plea...,Sabor,1364285.0,N,Y,29.383092,-98.376562,5.0,3,2019
20010,PAST SALE,78222,196499,2324.0,5662.0,2014.0,6.0,Sold,NaN,NaN,http://www.redfin.com/TX/San-Antonio/6006-Bear...,Sabor,1362872.0,N,Y,29.379346,-98.370269,7.0,3,2019
